# Install and Import Dependencies

In [1]:
!pip uninstall opencv-python-headless --yes

Found existing installation: opencv-python-headless 4.5.5.62
Uninstalling opencv-python-headless-4.5.5.62:
  Successfully uninstalled opencv-python-headless-4.5.5.62


In [2]:
!pip install mediapipe opencv-python-headless==4.5.1.48

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [3]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Make Detections

# Determining Joints

# Calculate Angles

# Curl Counter